## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV
from sklearn.preprocessing import normalize, OneHotEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import log_loss, accuracy_score, recall_score, make_scorer
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

## Reading Data

In [2]:
# reading CSVs
vehicle_df = pd.read_csv('../data/localdata/Traffic_Crashes_Vehicles.csv')
people_df = pd.read_csv('../data/localdata/Traffic_Crashes_People.csv')
crash_df = pd.read_csv('../data/localdata/Traffic_Crashes_Crashes.csv')

/Users/michaelromanski/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/michaelromanski/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (20,24,25,26,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
crash_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752960 entries, 0 to 752959
Data columns (total 49 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                752960 non-null  object 
 1   RD_NO                          748818 non-null  object 
 2   CRASH_DATE_EST_I               56786 non-null   object 
 3   CRASH_DATE                     752960 non-null  object 
 4   POSTED_SPEED_LIMIT             752960 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         752960 non-null  object 
 6   DEVICE_CONDITION               752960 non-null  object 
 7   WEATHER_CONDITION              752960 non-null  object 
 8   LIGHTING_CONDITION             752960 non-null  object 
 9   FIRST_CRASH_TYPE               752960 non-null  object 
 10  TRAFFICWAY_TYPE                752960 non-null  object 
 11  LANE_CNT                       199004 non-null  float64
 12  ALIGNMENT                     

## Data Cleaning

In [4]:
# Crashes DataFrame

crash_df.drop(columns=['WORK_ZONE_I', 'WORK_ZONE_TYPE', 'DOORING_I', 'WORKERS_PRESENT_I',
                                          'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'RD_NO', 'REPORT_TYPE', 'CRASH_DATE_EST_I'], axis=1, inplace=True)
crash_df.dropna(subset = ['INJURIES_TOTAL', 'LATITUDE', 'MOST_SEVERE_INJURY', 
                          'STREET_DIRECTION', 'BEAT_OF_OCCURRENCE'], axis=0, inplace=True)
crash_df.columns = crash_df.columns.str.title()
crash_df.columns = crash_df.columns.str.replace('_', ' ')

In [5]:
# Vehicles DataFrame

vehicle_df = vehicle_df[['CRASH_UNIT_ID', 'CRASH_RECORD_ID', 'CRASH_DATE', 'UNIT_NO', 'UNIT_TYPE',
                       'VEHICLE_YEAR', 'VEHICLE_USE', 'VEHICLE_TYPE', 'VEHICLE_DEFECT', 'MANEUVER', 'OCCUPANT_CNT',
                         'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I', 'AREA_04_I',
                       'AREA_05_I', 'AREA_06_I', 'AREA_07_I', 'AREA_08_I', 'AREA_09_I', 'AREA_10_I', 'AREA_11_I',
                       'AREA_12_I', 'AREA_99_I', 'FIRST_CONTACT_POINT']]
vehicle_df.dropna(subset = ['VEHICLE_USE', 'FIRST_CONTACT_POINT', 'UNIT_TYPE'], axis=0, inplace=True)
vehicle_df['VEHICLE_YEAR'].fillna(value = 'Unknown', inplace=True)

vehicle_df.columns = vehicle_df.columns.str.title()
vehicle_df.columns = vehicle_df.columns.str.replace('_', ' ')

In [6]:
# People DataFrame

people_df.drop(columns=['RD_NO', 'CELL_PHONE_USE', 'PEDPEDAL_ACTION', 
                        'PEDPEDAL_VISIBILITY', 'PEDPEDAL_LOCATION', 'SEAT_NO', 
                        'HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO', 'BAC_RESULT', 'BAC_RESULT VALUE', 
                        'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'CITY', 'STATE', 'ZIPCODE']
               , axis=1, inplace=True)

people_df.dropna(subset = ['AIRBAG_DEPLOYED', 'EJECTION', 'INJURY_CLASSIFICATION', 'VEHICLE_ID', 
                           'SAFETY_EQUIPMENT', 'SEX'], axis=0, inplace=True)

people_df.columns = people_df.columns.str.title()
people_df.columns = people_df.columns.str.replace('_', ' ')

## Feature Engineering Crash Score

In [7]:
# Injury Score

crash_df[['Injuries Total', 'Injuries Fatal', 'Injuries Incapacitating', 
            'Injuries Non Incapacitating', 'Injuries Reported Not Evident']][crash_df['Injuries Total']!=0][:50]

crash_df['total injured'] = crash_df['Injuries Fatal'] + crash_df['Injuries Incapacitating'] + crash_df['Injuries Non Incapacitating'] + crash_df['Injuries Reported Not Evident']

# Checking if added up columns in 'total injured' make up 'Injuries Total'
(crash_df['total injured'] == crash_df['Injuries Total']).value_counts()

# Injury Score column is a linear combination of the factors that make up Injuries Total simply multiplied by a constant depending on their severity
crash_df['Injury Score'] = crash_df['Injuries Fatal']*7 + crash_df['Injuries Incapacitating']*3 + crash_df['Injuries Non Incapacitating']*2 + crash_df['Injuries Reported Not Evident']

# Accounting Material Damage of Car and Surrounding Areas )
# Crash Score 

crash_df['Damage'].value_counts()
mapping = {'OVER $1,500': 3, '$501 - $1,500': 2, "$500 OR LESS": 1}
crash_df['Damage_ODE'] = crash_df['Damage'].map(mapping)
crash_df['Injury Score'] = crash_df['Injury Score'] + crash_df['Damage_ODE']*3
crash_df['Crash Score'] = crash_df['Injury Score']

## Injury Classification Column (Predictor)

In [8]:
# def fill_injury_class_rows(row):
#     if row["Injuries Fatal"] > 0 :
#         return 'Fatal'
#     elif row["Injuries Incapacitating"] > 0 :
#         return 'Incapacitating'
#     elif row['Injuries Non Incapacitating'] + row['Injuries Reported Not Evident'] > 0:
#         return 'Minor'
#     else:
#         return 'None Injured'

# crash_df['Injury Classification'] = crash_df.apply(fill_injury_class_rows, axis=1)

# crash_df['Injury Classification']

In [9]:
def fill_fatal_class_rows(row):
    if row["Injuries Fatal"] > 0 :
        return 'Fatal'
    else:
        return 'Not Fatal'

crash_df['Fatality Classification'] = crash_df.apply(fill_fatal_class_rows, axis=1)

In [10]:
crash_df['Fatality Classification'].value_counts(normalize=True)


Not Fatal    0.998912
Fatal        0.001088
Name: Fatality Classification, dtype: float64

In [11]:
crash_df['First Crash Type'].value_counts()

PARKED MOTOR VEHICLE            174074
REAR END                        168995
SIDESWIPE SAME DIRECTION        113131
TURNING                         106334
ANGLE                            81051
FIXED OBJECT                     34650
PEDESTRIAN                       17163
PEDALCYCLIST                     11205
SIDESWIPE OPPOSITE DIRECTION     10627
OTHER OBJECT                      7184
REAR TO FRONT                     6780
HEAD ON                           6386
REAR TO SIDE                      4036
OTHER NONCOLLISION                2287
REAR TO REAR                      1435
ANIMAL                             529
OVERTURNED                         446
TRAIN                               39
Name: First Crash Type, dtype: int64

In [12]:
crash_df['Prim Contributory Cause'].value_counts()

UNABLE TO DETERMINE                                                                 288602
FAILING TO YIELD RIGHT-OF-WAY                                                        81766
FOLLOWING TOO CLOSELY                                                                73678
NOT APPLICABLE                                                                       39190
IMPROPER OVERTAKING/PASSING                                                          36649
FAILING TO REDUCE SPEED TO AVOID CRASH                                               31683
IMPROPER BACKING                                                                     29952
IMPROPER LANE USAGE                                                                  26990
IMPROPER TURNING/NO SIGNAL                                                           24770
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  24671
DISREGARDING TRAFFIC SIGNALS                                                         14523

## Merging Vehicle and Crash DataFrames

In [13]:
vehicles_crashes_df = vehicle_df.merge(crash_df, on = 'Crash Record Id', how = 'inner')
vehicles_crashes_df.drop_duplicates(subset='Crash Record Id', inplace=True)

In [14]:
categorical = ['Weather Condition', 'Roadway Surface Cond', 'Road Defect', 'Alignment',
                       'Traffic Control Device', 'Device Condition', 'Crash Hour', 'Trafficway Type', 'Maneuver', 'Vehicle Defect',
               'Lighting Condition', 'First Crash Type', 'Prim Contributory Cause', 'Sec Contributory Cause'] #potentiall vehicle defects

ohe = OneHotEncoder(drop='first')
ohe.fit(vehicles_crashes_df[categorical])

categorical_encoded = pd.DataFrame(ohe.transform(vehicles_crashes_df[categorical]).todense(),
                               columns=ohe.get_feature_names())
# for Nick's version of OneHotEncoder, since its newer, the attribute is get_feature_names_out but get_feature_name for bobby and mike's version

In [15]:
categorical_encoded.shape

(745092, 238)

In [16]:
vehicles_crashes_df.shape

(745092, 70)

## Creating Train_Test_Split

In [17]:
#X = categorical_encoded
#y = vehicles_crashes_df['Fatal Classification']

#X_train, X_test, y_train, y_test = train_test_split(X, y)

## Creating Dummy Model

In [18]:
#Instantiate dummy model that will always predict majority class
#dummy_model = DummyClassifier(strategy="most_frequent", random_state = 42)
#dummy_model.fit(X_train, y_train)

#baseline score
#dummy_model.score(X_train, y_train)

In [19]:
#y_train.value_counts(normalize=True)

In [20]:
#can also grab cross_val mean to see how it’s performing
#cv_results = cross_val_score(dummy_model, X_train, y_train, cv=5)
#cv_results.mean()

## Testing Dummy Model

In [21]:
#dummy_model.score(X_test, y_test)

## Creating Logistic Regression Model

In [22]:
# logreg = LogisticRegression(random_state=42, max_iter = 1000)
# log_model = logreg.fit(X_train, y_train)

# y_hat_train = log_model.predict(X_train)
# y_hat_test = log_model.predict(X_test)

In [23]:
#plot_confusion_matrix(log_model, X_train, y_train);

In [24]:
# print("Training Score: ", log_model.score(X_train, y_train))
# print("Testing Score: ", log_model.score(X_test, y_test))

## SMOTE

In [25]:
#CAUTION! This code takes about 20 minutes to run.
# # Instantiate our SMOTE
# sm = SMOTE(random_state=42)
# # Fit and resample on the training data
# X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)

In [26]:
#CAUTION! This code takes about 2 hours to run. 

# #Run Log Model on resampled SMOTE data
# log_model = logreg.fit(X_train_smote, y_train_smote)

# print("Training Score: ", log_model.score(X_train_smote, y_train_smote))
# print("Testing Score: ", log_model.score(X_test, y_test))

In [27]:
#y_train_smote.value_counts()

In [28]:
#plot_confusion_matrix(log_model, X_train_smote, y_train_smote);

Looking at this confusion matrix, it appeared that the model was performing well when predicting Fatal and None Injured. As Vision Zero's goal is for zero fatalities, we decided to hone our focus on Fatal vs. Not Fatal.

## Decision Tree Model

In [29]:
# normal decision tree model
# clf = DecisionTreeClassifier(criterion='gini', random_state=42, )

# clf.fit(X_train, y_train)

# y_preds = clf.predict(X_test)

# print('Accuracy: ', accuracy_score(y_test, y_preds))

# smote decision tree model
# clf = DecisionTreeClassifier(criterion='gini', random_state=42, max_depth =10)

# clf.fit(X_train_smote, y_train_smote)

# y_preds = clf.predict(X_test)

# print('Accuracy: ', accuracy_score(y_test, y_preds))

## Creating Train_Test_Split for Fatal vs. Not Fatal

In [30]:
vehicles_crashes_df['Fatality Classification'].value_counts(normalize=True)

Not Fatal    0.998914
Fatal        0.001086
Name: Fatality Classification, dtype: float64

In [31]:
X = categorical_encoded
y = vehicles_crashes_df['Fatality Classification']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Creating Dummy Model

In [32]:
#Instantiate dummy model that will always predict majority class
dummy_model = DummyClassifier(strategy="most_frequent", random_state = 42)
dummy_model.fit(X_train, y_train)

y_true = y_train
y_pred = dummy_model.predict(X_train)
#baseline score
recall_score(y_true, y_pred, pos_label='Fatal')

0.0

#### Ensuring we are evaluating models based on recall.
False Negatives in this case means that our model predicted a crash to be Not Fatal when in actuality it was Fatal. As lives are of the upmost importance, we want to minimize that chance of a False Negative as much as possible. 

In [33]:
recall_scorer = make_scorer(recall_score, pos_label='Fatal')

In [34]:
cv_score_dummy = cross_val_score(estimator = dummy_model, X=X_train, y=y_train, scoring = recall_scorer, n_jobs=-1).mean()
cv_score_dummy

0.0

## Creating a Logistic Regression Model

In [35]:
#logreg = LogisticRegression(random_state=42, max_iter = 1000, class_weight = 'balanced')
#logreg.fit(X_train, y_train)
#y_true = y_train
#y_pred = log_model.predict(X_train)
#recall_score(y_true, y_pred, pos_label = 'Fatal')

NameError: name 'log_model' is not defined

In [38]:
logreg = LogisticRegression(random_state=42, max_iter = 2000, class_weight = 'balanced', penalty = 'none')
logreg.fit(X_train, y_train)
y_true = y_train
y_pred = logreg.predict(X_train)
recall_score(y_true, y_pred, pos_label = 'Fatal')

0.8611570247933884

In [ ]:
# Attempting to flip our confusion matrix so Fatal is positive label
cm = confusion_matrix(y_true, y_pred).T
cm

In [ ]:
plot_confusion_matrix(log_model, X_train, y_train, labels = ['Not Fatal', 'Fatal']);

In [ ]:
y_train

In [ ]:
log_model.classes_

In [ ]:
#cv_score_log = cross_val_score(estimator = log_model, X=X_train, y=y_train, scoring = recall_scorer, n_jobs=4).mean()
#cv_score_log

#### Logistic Regression Pipeline including SMOTE and Model

In [ ]:
#enter non-smoted data into pipeline

In [39]:
logpipe = Pipeline([('smote', SMOTE(random_state=42, sampling_strategy = 0.3)),
                    ('model', logreg)])
#logpipe.fit(X_train, y_train)

In [40]:
#Cross Validating LogReg with Pipeline
cv_score_logpipe = cross_val_score(estimator=logpipe, X=X_train, y=y_train, scoring = recall_scorer, n_jobs = -1).mean()
cv_score_logpipe

0.715702479338843

In [ ]:
plot_confusion_matrix(logpipe, X_train, y_train, labels = ['Not Fatal', 'Fatal']);

In [42]:
coefficients_table = logpipe.named_steps['model'].coef_
coefficients_table = coefficients_table.transpose()
coefficients_table = [x[0] for x in coefficients_table]
coefficients = pd.Series(list(coefficients_table), name = 'coefficients')
features = pd.Series(list(categorical_encoded), name = 'features')
coefs_df = pd.DataFrame([coefficients, features]).transpose()
coefs_df.sort_values(by='coefficients', inplace=True, ascending=False)
coefs_df.tail(15)

,coefficients,features
167,-10.9933,x12_DISTRACTION - FROM INSIDE VEHICLE
196,-11.2544,x12_UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE ...
164,-11.3092,x12_DISREGARDING STOP SIGN
197,-11.4053,"x12_VISION OBSCURED (SIGNS, TREE LIMBS, BUILDI..."
176,-11.6525,x12_FAILING TO REDUCE SPEED TO AVOID CRASH
190,-11.6571,x12_RELATED TO BUS STOP
170,-11.7487,x12_DRIVING ON WRONG SIDE/WRONG WAY
185,-11.7909,x12_NOT APPLICABLE
168,-11.9363,x12_DISTRACTION - FROM OUTSIDE VEHICLE
187,-12.0841,"x12_OPERATING VEHICLE IN ERRATIC, RECKLESS, CA..."


In [43]:
coefs_df.to_csv('../data/localdata/coefs.csv', index=True)

#### Most likely will delete as SMOTE is now done in pipeline ---- Using SMOTE to account for our class imbalance

In [ ]:
#Most likely will delete as SMOTE is done in pipeline now
sm = SMOTE(random_state=42, sampling_strategy = 0.3)

X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)



In [ ]:
#Most likely will delete as SMOTE is done in pipeline now
log_model_smote = logreg.fit(X_train_smote, y_train_smote)

y_true_smote = y_train_smote
y_pred_smote = log_model_smote.predict(X_train_smote)

print("Training recall: ", recall_score(y_true_smote, y_pred_smote, pos_label = 'Fatal'))

In [ ]:
#Most likely will delete as SMOTE is done in pipeline now
cv_scores_smote = cross_val_score(estimator = log_model_smote, X=X_train_smote, y=y_train_smote, 
                                  scoring = recall_scorer, n_jobs = -1).mean()
cv_scores_smote

In [ ]:
#Most likely will delete as SMOTE is done in pipeline now
log_model_smote.coef_

In [ ]:
# Grid Search
logreg = LogisticRegression(random_state=42, max_iter = 2000, n_jobs= -1, class_weight = 'balanced')

logpipe = Pipeline([('smote', SMOTE(random_state=42, n_jobs=-1)),
                    ('model', logreg)])

param_grid = {
    "smote__sampling_strategy": [0.3, 0.4, 0.5],
    "model__C": np.logspace(-4, 4, 4),
    "model__penalty": ['l1', 'none']
}

search = GridSearchCV(logpipe, param_grid, scoring= recall_scorer, n_jobs=-1)
search.fit(X_train, y_train)

print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

## Decision Tree Model

In [ ]:
#normal decision tree model
clf = DecisionTreeClassifier(criterion='gini', random_state=42, class_weight = 'balanced', max_depth = 24, min_samples_split = 5000)

clf.fit(X_train, y_train)

y_preds = clf.predict(X_train)

print('Recall: ', recall_score(y_train, y_preds, pos_label = 'Fatal'))

In [ ]:
#Cross Validating clf
cv_score_clf = cross_val_score(estimator = clf, X=X_train, y=y_train, scoring = recall_scorer, n_jobs=-1).mean()
cv_score_clf

#### Decision Tree Pipeline for clf with SMOTE and Model

In [ ]:
#Create Pipeline
clfpipe = Pipeline([('smote', SMOTE(random_state=42, sampling_strategy = 0.3)),
                    ('model', clf)])

In [ ]:
#Cross Validating Decision Tree with pipeline
cv_score_clfpipe = cross_val_score(estimator=clfpipe, X=X_train, y=y_train, scoring = recall_scorer).mean()
cv_score_clfpipe

## Decision Tree Revised 1 (clf_1)

In [ ]:
#Instantiating Decision Tree with new parameters to increase recall score
clf_1 = DecisionTreeClassifier(criterion='gini', random_state=42, class_weight = 'balanced', max_depth = 24,
                                   min_samples_split = 2500)

clf_1.fit(X_train, y_train)

y_preds = clf_1.predict(X_train)

print('Recall: ', recall_score(y_train, y_preds, pos_label = 'Fatal'))

#### Decision Tree Pipeline for clf_1 with SMOTE and Model

In [ ]:
#Create Pipeline with new model
clf_1pipe = Pipeline([('smote', SMOTE(random_state=42, sampling_strategy = 0.3)),
                    ('model', clf_1)])

In [ ]:
#Cross Validating Decision Tree with pipeline
cv_score_clf_1pipe = cross_val_score(estimator=clf_1pipe, X=X_train, y=y_train, scoring = recall_scorer).mean()
cv_score_clf_1pipe

### Organizing Feature Importance Data for further analysis

In [ ]:
feature_importance = clf_1.feature_importances_
feature_importance

In [ ]:
categorical_encoded.columns

In [ ]:
feature_importances = pd.Series(list(feature_importance), name = 'feature_importance')
features = pd.Series(list(categorical_encoded), name = 'features')


In [ ]:
features

In [ ]:
feature_importances

In [ ]:
features_df = pd.DataFrame([feature_importances, features]).transpose()

In [ ]:
features_df

In [ ]:
features_df.sort_values(by='feature_importance', inplace=True, ascending=False)

In [ ]:
features_df.head(15)

## Feature Importance Analysis

In [47]:
#Check to see if indices align
X.head()

,x0_BLOWING SNOW,x0_CLEAR,x0_CLOUDY/OVERCAST,x0_FOG/SMOKE/HAZE,x0_FREEZING RAIN/DRIZZLE,x0_OTHER,x0_RAIN,x0_SEVERE CROSS WIND GATE,x0_SLEET/HAIL,x0_SNOW,...,x13_PHYSICAL CONDITION OF DRIVER,x13_RELATED TO BUS STOP,x13_ROAD CONSTRUCTION/MAINTENANCE,x13_ROAD ENGINEERING/SURFACE/MARKING DEFECTS,x13_TEXTING,x13_TURNING RIGHT ON RED,x13_UNABLE TO DETERMINE,x13_UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED),"x13_VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)",x13_WEATHER
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
#Check to see if indices align
y.head()

0    Not Fatal
2    Not Fatal
5    Not Fatal
7    Not Fatal
9    Not Fatal
Name: Fatality Classification, dtype: object

In [49]:
#reset y index so can merge with X on the index
y.reset_index(drop = True, inplace = True)

In [50]:
#Confirm reset was successful
y.head()

0    Not Fatal
1    Not Fatal
2    Not Fatal
3    Not Fatal
4    Not Fatal
Name: Fatality Classification, dtype: object

In [51]:
#Merge dfs to be able to perform calculations with all data together
categorical_and_fatal_df = X.merge(y, left_index = True, right_index = True, how = 'right')

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
#Confirm merge was successful. Should have same # of rows as before with one more column than X
categorical_and_fatal_df.shape

In [45]:
#Function takes in a dataframe and relevant columns.
#Function loops through each column, calculating the percentage of Fatalities for each column's involvment in crashes
#Function outputs a dataframe with column names and their Fatality Percentage in descending order
def calc_percentage(df, columns):
    
    percent_dict = {}
    
    for col in columns:    
        #Calculate percentage
        col_perc = len(df.loc[(df[col]==1) & (df['Fatality Classification']=='Fatal')]) / len(df.loc[df[col]==1])
        #Add percentage to dictionary
        percent_dict[col] = col_perc
    #Convert dictionary to DataFrame
    percent_df = pd.DataFrame.from_dict(percent_dict, orient='index', columns=['Fatality Percentage'])
    percent_df.sort_values(by = 'Fatality Percentage', ascending = False, inplace = True)
    
    return percent_df

In [52]:
relevant_columns = features_df['features'].unique()[:15].tolist() #select columns to analyze the fatal percentage of
feature_fatal_percent_df = calc_percentage(categorical_and_fatal_df, relevant_columns)
feature_fatal_percent_df

NameError: name 'features_df' is not defined

In [60]:
relevant_columns = coefs_df['features'].unique()[-45:].tolist() #select columns to analyze the fatal percentage of
coef_fatal_percent_df = calc_percentage(categorical_and_fatal_df, relevant_columns)
coef_fatal_percent_df

,Fatality Percentage
x12_PHYSICAL CONDITION OF DRIVER,0.016615
x8_DRIVERLESS,0.016129
x11_PEDESTRIAN,0.011919
x12_EXCEEDING AUTHORIZED SPEED LIMIT,0.010736
x4_SCHOOL ZONE,0.010714
x11_ANIMAL,0.009452
x11_OVERTURNED,0.009091
x4_PEDESTRIAN CROSSING SIGN,0.006711
x4_RAILROAD CROSSING GATE,0.006211
x12_DRIVING ON WRONG SIDE/WRONG WAY,0.005339


In [ ]:
#x6 is Crash Hour, 1 corresponding to 1pm and 21 corresponding to 9pm
vehicles_crashes_df['Crash Hour'].value_counts()

In [ ]:
#Group by index for easier group analysis
grouped_feature_fatal_percent_df = feature_fatal_percent_df.sort_index()
grouped_feature_fatal_percent_df

In [61]:
#Output df to csv for graph making in Tableau
coef_fatal_percent_df.to_csv('../data/localdata/coef_fatal_percent.csv', index=True)
#grouped_feature_fatal_percent_df.to_csv('../data/localdata/grouped_feature_fatal_percent.csv', index=True)

## Testing Data

In [ ]:
y_preds_test = clf_1pipe.predict(X_test)
print('Recall: ', recall_score(y_test, y_preds_test, pos_label = 'Fatal'))

In [ ]:
y_pred_test = logpipe.predict(X_test)

print("Training recall: ", recall_score(y_test, y_pred_test, pos_label = 'Fatal'))

In [ ]:
plot_confusion_matrix(logpipe, y_test, y_pred_test, labels = ['Not Fatal', 'Fatal']);